# A&E Data Transformation and Preprocessing

This notebook contains the data transformation process for the A&E (Accident & Emergency) optimization project for Public Health Scotland.

## Required Data

For this analysis, we need:
- **Hospital locations**: A map of hospitals in Scotalnd
- **Patient distribution**: Patient data mapped by postcode/datazone
- **Wait times**: Waiting times for each patient
- **Accessibility data**: Driving/travel times for each patient to the facilities

## Workflow

1. Load and clean population data
2. Process hospital location data
3. Analyze A&E activity data
4. Merge datasets and create spatial joins
5. Visualize population distribution

## 1. Import Required Libraries

We'll use the following libraries:
- **pandas**: For data manipulation and analysis
- **geopandas**: For spatial data operations
- **matplotlib**: For data visualization
- **geodatasets**: For accessing geographic datasets

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geodatasets

## 2. Load and Process Population Data

In this section, we'll:
1. Load the population data from the CSV file
2. Filter the data to keep only the relevant information
3. Transform the age distribution data to a more usable format

In [ ]:
# Load the population data
data_pop = pd.read_csv("../../data/dz2011-pop-est_21112024.csv")
data_pop.head()

### 2.1 Filter Population Data

We need to filter the population dataset to:
- Keep only data from the year 2022
- Select records with "All" sex to get total population
- Remove rows with data quality issues (missing DataZoneQF)
- Drop unnecessary columns

In [ ]:
# Filter for year 2022, sex "All", and no DataZoneQF (good quality data)
data_pop = data_pop[
    (data_pop.Year == 2022) &
    (data_pop.Sex  == "All") &
    (data_pop.DataZoneQF.isna())
].copy()

# Remove unnecessary columns: flags, sex, and total population
data_pop.drop(columns=["DataZoneQF", "Sex", "SexQF", "AllAges"], inplace=True)

print(f"Number of datazones after filtering: {len(data_pop)}")
data_pop.head()

### 2.2 Transform Age Data

The age data is currently in wide format, with separate columns for each age group. 
We'll transform it to long format for easier analysis:
1. Melt the dataframe to convert age columns to rows
2. Convert age values to numeric format
3. Handle 90+ ages appropriately

In [ ]:
# Pivot age columns into long format
age_cols = [c for c in data_pop.columns if c.startswith("Age")]
data_pop = data_pop.melt(
    id_vars=[c for c in data_pop.columns if c not in age_cols],
    value_vars=age_cols,
    var_name="Age",
    value_name="Population"
)

# Convert Age to numeric, set 90+ as 90
data_pop['Age'] = data_pop['Age'].str.replace('Age', '')
data_pop['Age'] = pd.to_numeric(data_pop['Age'], errors='coerce')
data_pop['Age'].fillna(90, inplace=True)

print(f"Number of rows after transformation: {len(data_pop)}")
data_pop.head()

### 2.3 Check for Missing Values

Let's verify the data quality by checking for missing values in the population dataset:

In [ ]:
# Check for missing values in each column
print("📊 Number of missing values per column:")
print(data_pop.isna().sum())
print()

# Calculate percentage of missing values
print("📉 Percentage of missing values per column:")
print((data_pop.isna().mean() * 100).round(2))
print()

# Count rows with at least one missing value
nan_rows = data_pop.isna().any(axis=1).sum()
print(f"⚠️ Rows with at least one missing value: {nan_rows} out of {len(data_pop)}")
print()

## 3. Load and Process Hospital Data

Now we'll load the hospital dataset containing information about healthcare facilities:
1. Load the CSV file with hospital information
2. Remove unnecessary address details
3. Check data quality

In [ ]:
# Load hospital data
data_hospital = pd.read_csv("../../data/hospitals.csv")

# Drop address line columns (not needed for analysis)
addr_cols = [c for c in data_hospital.columns if c.startswith("AddressLine")]
data_hospital.drop(columns=addr_cols, inplace=True)

print(f"Number of hospitals: {len(data_hospital)}")
data_hospital.head()

### 3.1 Check for Missing Values in Hospital Data

Let's check the quality of the hospital dataset by examining missing values:

In [ ]:
# Check for missing values in each column
print("📊 Number of missing values per column:")
print(data_hospital.isna().sum())
print()

# Calculate percentage of missing values
print("📉 Percentage of missing values per column:")
print((data_hospital.isna().mean() * 100).round(2))
print()

# Count rows with at least one missing value
nan_rows = data_hospital.isna().any(axis=1).sum()
print(f"⚠️ Rows with at least one missing value: {nan_rows} out of {len(data_hospital)}")
print()

## 4. Load and Clean A&E Activity Data

Next, we'll process the A&E activity data:
1. Load the monthly activity data
2. Filter to keep only the aggregate records (AttendanceCategory = "All")
3. Remove unnecessary columns like percentages and quality flags
4. Prepare the data for analysis

In [ ]:
# Load A&E activity data
data_activity = pd.read_csv("../../data/monthly_ae_activity_202505.csv")

# Keep only records with AttendanceCategory = "All" (aggregate data)
data_activity = data_activity[data_activity.AttendanceCategory == "All"].copy()

# Drop unnecessary columns: attendance category, percentage columns, quality flags
to_drop = [c for c in data_activity.columns
           if c == 'AttendanceCategory'
           or 'Percentage' in c
           or c.endswith('QF')]

# Add more columns to drop (episode-level data that we don't need)
to_drop += [
    'NumberOfAttendancesEpisode',
    'NumberWithin4HoursEpisode',
    'NumberOver4HoursEpisode',
    'NumberOver8HoursEpisode',
    'NumberOver12HoursEpisode'
]

# Drop the columns
data_activity.drop(columns=to_drop, inplace=True)

# Display results
print(f"✅ Remaining rows: {len(data_activity)}")
print(f"📦 Remaining columns: {list(data_activity.columns)}")
data_activity.head()

### 4.1 Check for Missing Values in A&E Activity Data

Let's examine the quality of the A&E activity dataset:

In [ ]:
# Check for missing values in each column
print("📊 Number of missing values per column:")
print(data_activity.isna().sum())
print()

# Calculate percentage of missing values
print("📉 Percentage of missing values per column:")
print((data_activity.isna().mean() * 100).round(2))
print()

# Count rows with at least one missing value
nan_rows = data_activity.isna().any(axis=1).sum()
print(f"⚠️ Rows with at least one missing value: {nan_rows} out of {len(data_activity)}")
print()

## A&E Data Processing

After examining the missing values, we need to process the A&E activity data to prepare it for our analysis. This involves:

1. Converting date columns to the proper datetime format
2. Filtering the data by department type (ED or MIU)
3. Aggregating metrics by location and time period
4. Creating useful derived metrics for our analysis

Let's start by handling the date columns and examining the department types in our dataset.

### 4.2 Patient Data Generation

In this section, we expand the aggregate A&E data to create individual patient records. This is important for our analysis as it allows us to:

1. Model individual waiting times rather than just using aggregates
2. Apply different filters and segmentations to the data
3. Create more detailed visualizations

The process involves:
- Converting date columns to proper datetime format
- Identifying different department types (ED vs MIU)
- Creating synthetic patient records based on the aggregate counts
- Assigning reasonable waiting times:
  - For patients seen within 4 hours: random time between 1-240 minutes
  - For patients seen after 4 hours: random time between 241-720 minutes

This synthetic patient-level dataset will enable us to perform more detailed analyses while preserving the statistical properties of the original data.

In [ ]:
import numpy as np
from tqdm import tqdm  # Progress bar

# Create a copy of the original dataset
df = data_activity.copy()

# Show available columns
print("Available columns in data_activity:")
print(df.columns.tolist())
print()

# Convert Month column to datetime format
df["Month"] = pd.to_datetime(df["Month"], format="%Y%m")

# Display the unique department types in the dataset
print("Department types in the dataset:")
print(df["DepartmentType"].unique())
print()

# Fill any NaN values with 0
df[['NumberWithin4HoursAll', 'NumberOver4HoursAll', 'NumberOfAttendancesAll']] = \
    df[['NumberWithin4HoursAll', 'NumberOver4HoursAll', 'NumberOfAttendancesAll']].fillna(0).astype(int)

# List to collect expanded patient rows
patient_rows = []

# Initialize progress bar
print("⏳ Generating individual patient records...")
for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    total = row['NumberOfAttendancesAll']
    n_within = row['NumberWithin4HoursAll']
    n_over = row['NumberOver4HoursAll']

    # Patients seen within 4 hours
    if n_within > 0:
        wait_times_within = np.random.uniform(1, 240, size=n_within).astype(int)
        for wt in wait_times_within:
            new_row = row.drop(['NumberWithin4HoursAll', 'NumberOver4HoursAll', 'NumberOfAttendancesAll']).copy()
            new_row['WaitingTime'] = wt
            patient_rows.append(new_row)

    # Patients seen after 4 hours
    if n_over > 0:
        wait_times_over = np.random.uniform(241, 720, size=n_over).astype(int)
        for wt in wait_times_over:
            new_row = row.drop(['NumberWithin4HoursAll', 'NumberOver4HoursAll', 'NumberOfAttendancesAll']).copy()
            new_row['WaitingTime'] = wt
            patient_rows.append(new_row)

# Build the final DataFrame
print("📦 Building the final DataFrame...")
df_patients = pd.DataFrame(patient_rows)

# Final feedback
print(f"✅ Generation completed: {len(df_patients):,} total patients.")
df_patients.head()

## 5. Data Integration

Now we need to combine our datasets to create a comprehensive view of the A&E system:

1. Merge A&E activity data with hospital information
2. Check for any missing matches (facilities that don't appear in both datasets)
3. Prepare the data for spatial analysis

In [ ]:
# Merge A&E activity data with hospital information
merged_data = pd.merge(
    data_activity,
    data_hospital,
    left_on="TreatmentLocation",
    right_on="HospitalCode",
    how="left"
)
print(f"✅ Number of records after merging: {len(merged_data)}")
merged_data.head()

### 5.1 Check for Missing Values in Merged Data

After merging the datasets, we need to check for missing values to ensure data quality. This helps us identify:
- A&E facilities that might be missing from our hospital dataset
- Missing location information that could affect our spatial analysis
- Other potential data quality issues that need addressing

In [ ]:
# Check for missing values in each column
print("📊 Number of missing values per column:")
print(merged_data.isna().sum())
print()

# Calculate percentage of missing values
print("📉 Percentage of missing values per column:")
print((merged_data.isna().mean() * 100).round(2))
print()

## 6. A&E Performance Analysis

Now let's analyze the performance of each A&E facility based on our synthetic patient dataset. We'll:

1. Group data by treatment location
2. Calculate key performance metrics:
   - Total number of patients
   - Average waiting time (in minutes)
   - Median waiting time (in minutes)
   - Maximum waiting time (in minutes)
   - Percentage of patients seen within 4 hours (the NHS target)
3. Sort facilities by their average wait time to identify best and worst performers

In [ ]:
# Group data by location and calculate metrics
location_summary = df_patients.groupby('TreatmentLocation').agg(
    total_patients=('WaitingTime', 'count'),
    avg_wait_mins=('WaitingTime', 'mean'),
    median_wait_mins=('WaitingTime', 'median'),
    max_wait_mins=('WaitingTime', 'max'),
    pct_within_4h=('WaitingTime', lambda x: (x <= 240).mean() * 100)
).reset_index()

# Sort by average wait time
location_summary = location_summary.sort_values('avg_wait_mins')

print(f"Summary statistics for {len(location_summary)} facilities:")
location_summary.head(10)

### 6.1 Department Type Comparison

A key aspect of our analysis is understanding the difference in performance between Emergency Departments (ED) and Minor Injury Units (MIU). This comparison helps us:

1. Identify which type of facility provides faster service
2. Understand the patient distribution between facility types
3. Evaluate performance against the 4-hour target for each facility type

Emergency Departments typically handle more severe cases, while Minor Injury Units focus on less critical injuries. We expect to see different patterns in waiting times between these facility types.

In [ ]:
# Filter patient data for specific department types
ed_patients = df_patients[df_patients.department_type == 'Emergency Department'].copy()
miu_patients = df_patients[df_patients.department_type == 'Minor Injury Unit'].copy()

# Compare performance between department types
dept_comparison = pd.DataFrame({
    'Department Type': ['Emergency Department', 'Minor Injury Unit'],
    'Number of Patients': [len(ed_patients), len(miu_patients)],
    'Avg Wait (mins)': [ed_patients.WaitingTime.mean(), miu_patients.WaitingTime.mean()],
    'Median Wait (mins)': [ed_patients.WaitingTime.median(), miu_patients.WaitingTime.median()],
    '% Within 4h': [(ed_patients.WaitingTime <= 240).mean() * 100, 
                    (miu_patients.WaitingTime <= 240).mean() * 100]
})

print("Comparison of department types:")
dept_comparison

## 7. Time Series Analysis

Understanding how A&E performance changes over time is critical for identifying:
- Seasonal patterns in patient volumes
- Trends in waiting times
- Changes in performance against the 4-hour target

Let's create a time series analysis of our synthetic patient data to identify these patterns:

In [ ]:
# Create a time series of average wait times by month
monthly_trend = df_patients.groupby(pd.Grouper(key='month', freq='M')).agg(
    avg_wait=('WaitingTime', 'mean'),
    median_wait=('WaitingTime', 'median'),
    pct_within_4h=('WaitingTime', lambda x: (x <= 240).mean() * 100),
    num_patients=('WaitingTime', 'count')
).reset_index()

# Display the first few rows
print(f"Monthly trends over {len(monthly_trend)} months:")
monthly_trend.head()

### 7.1 Visualizing Monthly Trends

Let's create visualizations to better understand how A&E performance varies over time:

1. **Top Chart**: Shows average and median wait times compared to the 4-hour target
2. **Bottom Chart**: Displays the percentage of patients seen within 4 hours compared to the NHS 95% target

These visualizations help identify:
- Seasonal patterns (e.g., winter pressures on A&E services)
- Overall trends in performance
- Periods where performance is significantly below target

In [ ]:
# Set up the figure for visualization
plt.figure(figsize=(12, 8))

# Plot average wait time over time
plt.subplot(2, 1, 1)
plt.plot(monthly_trend['month'], monthly_trend['avg_wait'], 'b-', label='Average Wait Time')
plt.plot(monthly_trend['month'], monthly_trend['median_wait'], 'g--', label='Median Wait Time')
plt.axhline(y=240, color='r', linestyle='--', label='4-Hour Target')
plt.title('Monthly Average and Median Wait Times', fontsize=14)
plt.ylabel('Time (minutes)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot percentage meeting 4-hour target over time
plt.subplot(2, 1, 2)
plt.plot(monthly_trend['month'], monthly_trend['pct_within_4h'], 'g-')
plt.fill_between(monthly_trend['month'], monthly_trend['pct_within_4h'], alpha=0.2, color='g')
plt.title('Percentage of Patients Seen Within 4 Hours', fontsize=14)
plt.ylabel('Percentage')
plt.axhline(y=95, color='r', linestyle='--', label='95% Target')
plt.legend(['% Within 4h', '95% Target'])
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../../results/images_dir/monthly_performance_trends.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Spatial Analysis

To understand the geographic distribution of healthcare facilities and identify potential areas for improvement, we need to perform spatial analysis:

1. Create a GeoDataFrame from our hospital data
2. Visualize facility locations on a map
3. Analyze the spatial distribution of wait times and performance

This spatial component will help us identify:
- Areas with limited A&E access
- Geographic patterns in wait times
- Potential locations for new facilities

In [ ]:
# Create a GeoDataFrame from hospital data with coordinates
gdf_hospitals = gpd.GeoDataFrame(
    data_hospital,
    geometry=gpd.points_from_xy(data_hospital.Longitude, data_hospital.Latitude),
    crs="EPSG:4326"
)

# Preview the GeoDataFrame
print(f"Created GeoDataFrame with {len(gdf_hospitals)} hospital locations")
gdf_hospitals.head()

### 8.1 Combining Performance Data with Geographic Information

To create meaningful spatial visualizations, we need to join our performance metrics with the geographic hospital data:

1. Merge the location summary with hospital GeoDataFrame
2. Handle missing values for facilities without activity data
3. Prepare the data for mapping

This allows us to visualize wait times and other metrics across geographic space, helping identify patterns that might not be apparent in tabular data.

In [ ]:
# Join performance metrics to hospital locations
gdf_hospitals = gdf_hospitals.merge(
    location_summary,
    left_on='HospitalCode',
    right_on='TreatmentLocation',
    how='left'
)

# Fill NaN values for facilities without activity data
gdf_hospitals['total_patients'] = gdf_hospitals['total_patients'].fillna(0)
gdf_hospitals['avg_wait_mins'] = gdf_hospitals['avg_wait_mins'].fillna(0)
gdf_hospitals['pct_within_4h'] = gdf_hospitals['pct_within_4h'].fillna(0)

print(f"Number of facilities with activity data: {(gdf_hospitals['total_patients'] > 0).sum()}")
print(f"Number of facilities without activity data: {(gdf_hospitals['total_patients'] == 0).sum()}")
print()

### 8.2 Creating the Final Dataset

After analyzing the merged data, we need to create a comprehensive dataset that combines all our data sources:
- A&E activity data
- Hospital location data
- Population data by datazone

This final dataset will be used for our spatial analysis and optimization models.

In [ ]:
# Create the final dataset by joining our datasets
# 1. First join merged_data (A&E activity + hospital info) with population data
final_data = pd.merge(
    merged_data,
    data_pop,
    on="DataZone",
    how="right"  # Use right join as in R's right_join
)

# 2. Filter for specific months (as done in the R script)
# Define the months to include (from 202406 to 202505)
year = ["202406", "202407", "202408", "202409", "202410",
        "202411", "202412", "202501", "202502", "202503",
        "202504", "202505"]

# Filter the final data to include only these months
final_data = final_data[final_data["Month"].isin(year)]

# 3. Save the final dataset to CSV
final_data.to_csv("../../data/FINAL_DATA.csv", index=False)

print(f"✅ Final dataset created with {len(final_data)} records")
final_data.head()

In [ ]:
# Check for missing values in each column
print("📊 Number of missing values per column:")
print(final_data.isna().sum())
print()

# Calculate percentage of missing values
print("📉 Percentage of missing values per column:")
print((final_data.isna().mean() * 100).round(2))
print()

# Count rows with at least one missing value
nan_rows = final_data.isna().any(axis=1).sum()
print(f"⚠️ Rows with at least one missing value: {nan_rows} out of {len(final_data)}")
print()

In [ ]:
# Display sample of the final dataset with key columns
print("Sample of the final dataset:")
final_data.head()

In [ ]:
# Spatial join with datazone boundaries
# Load the shapefile with datazone geometries
dz = gpd.read_file("../../data/shapefiles/SG_DataZone_Bdry_2011.shp")

# Filter to keep only datazones present in our final dataset
selected = dz[dz.DataZone.isin(final_data.DataZone)]

# Join the final data with spatial boundaries
final_sf = selected.merge(final_data, on="DataZone", how="left")

In [ ]:
final_sf.head()

In [ ]:
# Create a map of hospital locations with wait time information
fig, ax = plt.subplots(figsize=(12, 10))

# Get Scotland boundaries for context
world = gpd.read_file(geodatasets.get_path('naturalearth.land'))
uk = world[world['continent'] == 'Europe']
uk.plot(ax=ax, color='lightgray', edgecolor='gray')

# Plot hospitals with size proportional to patient volume and color by wait time
scatter = gdf_hospitals[gdf_hospitals.total_patients > 0].plot(
    ax=ax,
    column='avg_wait_mins',
    cmap='RdYlGn_r',  # Red (bad) to Green (good)
    markersize=gdf_hospitals['total_patients'] / 1000 + 10,
    legend=True,
    legend_kwds={'label': 'Average Wait Time (mins)'},
    edgecolor='black',
    linewidth=0.5
)

# Add labels for major facilities
for idx, row in gdf_hospitals[gdf_hospitals.total_patients > 50000].iterrows():
    plt.annotate(row['OrganisationName'], 
                 xy=(row.geometry.x, row.geometry.y),
                 xytext=(5, 5),
                 textcoords="offset points",
                 fontsize=8,
                 bbox={'facecolor': 'white', 'alpha': 0.8, 'pad': 1})

# Add title and customize map
plt.title('Hospital Locations with Average A&E Wait Times', fontsize=16)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.axis('equal')

# Save the map
plt.savefig('../../results/images_dir/hospital_wait_times_map.png', dpi=300, bbox_inches='tight')
plt.show()

### 8.2 Creating a Map of A&E Wait Times

To visualize the geographic distribution of wait times across Scotland, we'll create a map showing:

1. Hospital locations with symbols sized by patient volume
2. Color-coded markers representing average wait times (red for longer waits, green for shorter)
3. Labels for major facilities handling large patient volumes
4. Geographic context using Scotland's boundaries

This visualization helps stakeholders quickly identify:
- Geographic clusters of high wait times
- Areas that might be underserved
- Relationships between facility size and performance

In [ ]:
# Compute average population per data zone
geom_data = final_sf.dissolve(
    by='DataZone',
    aggfunc={'Population': 'mean'}
)

In [ ]:
# Reset index and ensure correct CRS (coordinate reference system)
geom_data = geom_data.reset_index()
geom_data = geom_data.set_geometry('geometry')
geom_data.crs = dz.crs  # Ensure the coordinate system is correct

# Create a map showing average population by data zone
fig, ax = plt.subplots(1, 1, figsize=(12, 14))
geom_data.plot(
    column='Population',
    cmap='viridis',        # Alternative options: 'OrRd', 'plasma', 'YlGnBu', etc.
    linewidth=0.1,
    edgecolor='gray',
    legend=True,
    ax=ax
)
ax.set_title("📍 Average Population per Data Zone (Scotland)", fontsize=16)
ax.set_axis_off()

# Save the visualization
plt.savefig('../../results/images_dir/population_by_datazone.png', dpi=300, bbox_inches='tight')
plt.show()

## Conclusion and Next Steps

In this notebook, we've:

1. **Processed population data** to understand the demographic distribution across data zones
2. **Analyzed hospital locations** to map healthcare facilities
3. **Transformed A&E activity data** into a synthetic patient-level dataset
4. **Calculated key performance metrics** including wait times and 4-hour target compliance
5. **Visualized temporal trends** in A&E performance
6. **Created spatial visualizations** showing the geographic distribution of wait times

### Next Steps

Based on this exploratory analysis, the next steps for our project include:

1. **Develop optimization models** to improve patient allocation across facilities
2. **Identify optimal locations** for potential new facilities
3. **Simulate changes** to evaluate how modifications might affect waiting times
4. **Create an interactive dashboard** for stakeholders to explore scenarios
5. **Develop recommendations** for Public Health Scotland to improve A&E performance

The results of this analysis provide a foundation for the optimization work in subsequent notebooks and will inform our final recommendations.